We're going to be building a structure inspired by the symmetry of the gyroid shape. Specifically, this will be part of space group 230 and shares symmetry with the achiral 2-srs network reported [here](https://onlinelibrary.wiley.com/doi/10.1002/adom.201800485)


In [1]:
import crystalbuilder.bilbao as cbb

single_gyr = cbb.SpaceGroup(230, points=[(0,0,0), (.5,.5,.5)])

The above cell imports the bilbao module and pulls the generators/symmetry operations from the server. With the initial points specified, the single_gyr object now contains all of the points in the unit cell.

We need to build this unit cell using the geometry functions in crystalbuilder's geometry module. This could probably be done with MEEP as well, but it would generally be tedious to transform all of the points into lattice coordinates for MPB. **Note that it would not be as tedious here because the cubic cell lattice vectors are parallel to the cartesian axes.**

In [2]:
import crystalbuilder.geometry as geo

a = 1

rad = .1*a

unit_cell = geo.SuperCell([])

for n in single_gyr.generated_points:
    sphere = geo.Sphere(center=n, radius=rad)
    unit_cell.add_structure(sphere)

# unit_cell.identify_structures()

In [3]:
import crystalbuilder.lattice as lat

lattice = lat.Lattice()

crystal = lattice.tile_geogeometry(unit_cell, 3, 3, 3)


In [ ]:
import crystalbuilder.convert as cv
import meep as mp
from meep import mpb

mat1 = mp.Medium(epsilon=8)
#mpb_geo = cv.geo_to_mpb(unit_cell, material=mat1, lattice=cv.to_mpb_lattice(lattice))
mpb_geo = mp.Sphere(.1, center=mp.Vector3(0,0,0), material=mat1)
geometry_lattice = mp.Lattice()

# geometry_lattice = cv.to_mpb_lattice(lattice)

# k_labels = list(single_gyr.kvec_dict.keys())
# k_points = list(single_gyr.kvec_dict.values())
# k_points = mp.interpolate(1, k_points)

k_points = [
    mp.Vector3(0, 0.5, 0.5),        # X
    mp.Vector3(0, 0.625, 0.375),    # U
    mp.Vector3(0, 0.5, 0),          # L
    mp.Vector3(0, 0, 0),            # Gamma
    mp.Vector3(0, 0.5, 0.5),        # X
    mp.Vector3(0.25, 0.75, 0.5),    # W
    mp.Vector3(0.375, 0.75, 0.375)  # K
]

k_points = mp.interpolate(2, k_points)

resolution = 8
num_bands = 10

ms = mpb.ModeSolver(
    geometry=[mpb_geo],
    geometry_lattice=geometry_lattice,
    k_points=k_points,
    resolution=resolution,
    num_bands=num_bands,
)

ms.run()



: 

In [ ]:
import matplotlib.pyplot as plt
### Plotting Dielectric Map ###
plt.figure()
md = mpb.MPBData(rectify=True, periods=1, resolution=64)
eps = ms.get_epsilon()
converted_eps = md.convert(eps)
print (converted_eps.shape)
print (geometry_lattice.size)
plt.imshow(converted_eps.T, interpolation='spline36', cmap='binary_r')
plt.axis('off')

plt.figure()
md = mpb.MPBData(rectify=True, periods=2, resolution=64)
eps = ms.get_epsilon()
converted_eps = md.convert(eps)
print (converted_eps.shape)
print (geometry_lattice.size)
plt.imshow(converted_eps.T, interpolation='spline36', cmap='binary_r')
plt.axis('off')

if save== True:
    plt.savefig('structure-hachaspinhall-d'+str(d)+'-r-'+str(r)+'-'+mode+'.pdf')
    plt.close()
else:
    plt.show()

### Plotting Band Structure ###
if plot_structure_only == False:
    crysfreqs = ms.all_freqs
    x = range(len(crysfreqs))
    figgy = plt.figure()
    plt.plot(x,crysfreqs, color='blue');
    points_in_between = (len(crysfreqs)-1) / (k_corners-1)
    tick_locs = [i*points_in_between for i in range(k_corners)]
    tick_labs = k_points_str
    plt.xticks(tick_locs,tick_labs)
    plt.ylabel('frequency (c/a)', size=16)
    plt.title("%s Band Structure for n=%.2f \n  d= %.4f, r1 = %.4f, r2= %.4f " %(mode, nmat, d, r1, r2))
    plt.grid(True)
    figax = plt.gca()
    figax.set_ylim([bandmin, bandmax])
    if air_bands == True:
        airfreqs = ms_air.all_freqs
        figax.plot(x,airfreqs, color='black', ls='--')

    if dielec_bands == True:
        dielecfreqs = ms_dielectric.all_freqs
        figax.plot(x,dielecfreqs, color='black', ls=':')

    if save== True:
        plt.savefig('bands-hachaspinhall-d'+str(d)+'-r1-'+str(r1)+'-'+str(r2)+'-'+mode+'.pdf')
        plt.close()
    else:
        plt.show()

    plt.figure()
    crysfreqs_thz = 3e8*(crysfreqs/(a*1e-6))/1e12
    plt.plot(x, crysfreqs_thz, color='blue')
    ax1 = plt.gca()
    plt.xticks(tick_locs,tick_labs)
    plt.ylabel('frequency (THz)', size=16)
    plt.title("%s Band Structure for n=%.2f \n  d= %.4f, r1 = %.4f, r2= %.4f " %(mode, nmat, d, r1, r2))
    plt.grid(True)
    if air_bands == True:
        airfreqs = ms_air.all_freqs
        ax1.plot(x,airfreqs, color='black', ls='--')

    if dielec_bands == True:
        dielecfreqs = ms_dielectric.all_freqs
        ax1.plot(x,dielecfreqs, color='black', ls=':')
    
    plt.show()